# LCEL 인터페이스
- 사용자가 다양한 chain을 만들 수 있도록 langchaing의 대부분의 component들은 Runnable Protocol을 구현하고 있다. 
- 이는 chat models, LLMs, output parsers, retrievers, prompt templates 등 다양한 요소들이 Runnable 프로토콜을 구현하고 있다. 
- Runnable Protocol을 따르는 요소들로 쉽게 사용자가 자신만의 chain을 구성할 수 있다.
# Runnable
- Runnable은 invoke, batch, stream, transform, 그리고 compose되는 작업을 의미하며, 다음 함수들을 포함하고 있다.
    - `invoke`: 입력에 대해 체인을 호출
    - `stream`: 응답 청크를 스트리밍
    - `batch`: 입력 리스트에 대해 체인을 호출 -> 내부적으로 `invoke`를 thread pool executor로 병렬적으로 실행시킨다
- 위의 함수들에 대응되는 비동기 함수도 제공하고 있으며, 앞에 "a"가 붙은 이름이다.
    - `ainvoke`: invoke 비동기
    - `astream`: stream 비동기
    - `abatch`: batch 비동기
    - `astream_log`: 최종 응답 뿐만아니라 chain의 중간 단계를 스트리밍
    - `astream_events`: chain에서 발생하는 event에 대한 스트리밍 (beta)
- 모든 runnable은 입력과 출력 스키마를 노출시키고 Pydantic 모델로 자동생성된다.
    - `input_schema`: Runnable의 입력 Pydantic 모델
    - `output_schema`: Runnable의 출력 Pydantic 모델

더 자세한 내용은 [공식문서](https://python.langchain.com/v0.1/docs/expression_language/interface/)를 참고

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


In [2]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    # max_new_tokens=8,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface.chat_models import huggingface import ChatHuggingFace`.
  warn_deprecated(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Explain about {topic} in three sentences.")
chain = prompt | model

In [8]:
# chain의 가장 앞단의 입력이 chain의 input_schema가 된다.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [9]:
# chain의 가장 마지막 요소의 출력이 chain의 output_schema가 된다.
chain.output_schema.schema()

{'title': 'ChatHuggingFaceOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'p

아래와 같이 JSON으로 입력 스키마를 직접 정의할 수도 있다.

In [11]:
import json

# 스키마를 출력하는 함수를 정의합니다.
def print_schema(schema):
    # 스키마를 JSON 형식으로 출력합니다.
    print(json.dumps(schema, indent=4))

# 입력 스키마를 정의합니다.
# TODO: 유즈케이스 찾아보기
input_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "integer", "minimum": 0},
    },
    "required": ["name", "age"],
}

# 스키마를 출력하는 함수를 호출합니다.
print_schema(input_schema)

{
    "type": "object",
    "properties": {
        "name": {
            "type": "string"
        },
        "age": {
            "type": "integer",
            "minimum": 0
        }
    },
    "required": [
        "name",
        "age"
    ]
}


# Stream
- Runnable의 `stream`함수를 이용하여 chain의 출력을 스트리밍 할 수 있다.
- `content`은 stream 데이터의 내용, `end=""`은 출력 후 줄바꿈 하지 않도록 하는 것이고, `flush=True`는 출력 버퍼를 즉시 비워서 바로 출력하는 설정이다.

In [17]:
for s in chain.stream({"topic": "Llama3"}):
    print(s.content, end="", flush=True)

I apologize, but I couldn't find any information on "Llama3". It's possible that it's a fictional or non-existent entity, or it could be a misspelling or variation of a different term. If you could provide more context or clarify what you mean by "Llama3", I'd be happy to try and assist you further.<|eot_id|>

# invoke
- invoke 메서드는 chain의 입력을 받아서 chain을 실행시켜 출력을 반환한다.

In [18]:
chain.invoke({"topic": "ChatGPT"})

AIMessage(content='ChatGPT is a revolutionary AI chatbot developed by Meta AI that uses natural language processing (NLP) to understand and respond to human input in a conversational manner. It is trained on a massive dataset of text from the internet and can generate human-like responses to a wide range of questions and topics, from simple queries to complex discussions. ChatGPT is designed to be a helpful tool for people, allowing them to have a conversation with a machine that feels natural and intuitive, and has the potential to be used in a variety of applications, such as customer service, language translation, and content creation.', id='run-44b10eda-17d0-4221-9868-c48a0993ee4a-0')

# batch
- 여러 입력을 받아서 병렬적으로 처리.
- apply와 같은 역할
- `max_concurrency` 매개변수를 사용하여 동시 요청 수를 제한할 수도 있다.

In [19]:
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

[AIMessage(content='ChatGPT is a revolutionary AI chatbot developed by Meta AI that uses natural language processing (NLP) to understand and respond to human input in a conversational manner. It is trained on a massive dataset of text from the internet and can generate human-like responses to a wide range of questions and topics, from simple queries to complex discussions. ChatGPT is designed to be a helpful tool for people, allowing them to have a conversation with a machine that feels natural and intuitive, and has the potential to be used in a variety of applications, such as customer service, language translation, and content creation.', id='run-1ec65202-59b6-41dc-aeb7-8b7bcf9b83a2-0'),
 AIMessage(content='Instagram is a social media platform where users can share photos and videos with their followers, as well as engage with others by liking and commenting on their content. The platform is known for its visual-centric approach, with users able to share moments from their daily liv

In [20]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

[AIMessage(content='ChatGPT is a revolutionary AI chatbot developed by Meta AI that uses natural language processing (NLP) to understand and respond to human input in a conversational manner. It is trained on a massive dataset of text from the internet and can generate human-like responses to a wide range of questions and topics, from simple queries to complex discussions. ChatGPT is designed to be a helpful tool for people, allowing them to have a conversation with a machine that feels natural and intuitive, and has the potential to be used in a variety of applications, such as customer service, language translation, and content creation.', id='run-2b3dbb8d-61bc-441a-84f6-0e89e2fe658c-0'),
 AIMessage(content='Instagram is a social media platform where users can share photos and videos with their followers, as well as engage with others by liking and commenting on their content. The platform is known for its visual-centric approach, with users able to share moments from their daily liv

In [22]:
# astream: 비동기 스트림 생성
async for s in chain.astream({"topic": "Youtube"}):
    print(s.content, end="", flush=True)

YouTube is a video-sharing platform where users can upload, share, and view videos with others. The platform allows users to create their own channels and upload videos on a variety of topics, including music, educational content, vlogs, and more. With over 2 billion monthly active users, YouTube is one of the most popular websites on the internet, offering a vast array of content and a platform for creators to share their ideas and connect with audiences worldwide.<|eot_id|>

In [24]:
# ainvoke: 주어진 입력으로 비동기적으로 작업을 처리
await chain.ainvoke({"topic": "NVDA"})

AIMessage(content='NVDA (NonVisual Desktop Access) is a free and open-source screen reader software that allows visually impaired individuals to access and interact with their computers. It reads aloud the text on the screen, including menus, buttons, and other graphical elements, allowing users to navigate and use their computers independently. NVDA is compatible with a wide range of operating systems, including Windows, and is widely used by individuals who are blind or have low vision, as well as by organizations and businesses that provide assistive technology services.', id='run-a417d4cb-c526-4bde-a571-2091ec9e5025-0')

In [25]:
# abatch: 비동기적으로 입력 목록을 병렬로 처리
await chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

[AIMessage(content='YouTube is a video-sharing platform where users can upload, share, and view videos with others. Founded in 2005, YouTube has become one of the most popular websites on the internet, with over 2 billion monthly active users and over 5 billion videos viewed every day. On YouTube, users can create their own channels, upload videos, and monetize their content through ads, sponsorships, and merchandise sales, making it a significant platform for creators and influencers to build their online presence and earn a living.', id='run-ff128aa5-b0b4-45ad-811d-5515fbeb1717-0'),
 AIMessage(content='Instagram is a social media platform where users can share photos and videos with their followers, as well as engage with others by liking and commenting on their content. The platform is known for its visual-centric approach, with users able to share moments from their daily lives, showcase their creativity, and connect with others who share similar interests. With over a billion acti

# astream_log
- chain의 중간 단계에서 발생하는 모든 이벤트에 대한 세분화된 로그를 스트리밍할 수 있다.
- 로그 포멧은 [`LogEntry`와 `RunState` 클래스](https://python.langchain.com/v0.1/docs/expression_language/interface/#async-stream-intermediate-steps)에 대한 [JSONPatch 표준](https://jsonpatch.com/)을 기반으로 한다.
-  `astream_log`는 기본적으로 RunState에 차이만 JSONPatch 형식으로 출력해주지만, `diff=False` 옵션을 추가하면 RunState 전체 내용을 출력한다.
- 하지만 로그를 파싱하는데 많이 노력이 요하므로, astream_evens API를 만들었다.
- 아래 예제를 실행시키기 위해 아래 명령어로 패키지를 추가 설치해야 한다.
    ```bash
    pip install sentence-transformers
    conda install -c pytorch faiss-gpu
    ```


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["Teddy lives in South Korea"], embedding=HuggingFaceEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log(
    "Where does Teddy is living?", include_names=["Docs"], diff=False,
):
    print("-" * 40)
    print(chunk)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


----------------------------------------
RunLog({'final_output': None,
 'id': 'e220fbe5-e4aa-4238-ba35-a4520a291d4f',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------
RunLog({'final_output': None,
 'id': 'e220fbe5-e4aa-4238-ba35-a4520a291d4f',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': '8dee5e9c-bf69-4b23-83f7-84c491e83319',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2024-05-28T11:29:13.307+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['map:key:context',
                            'FAISS',
                            'HuggingFaceEmbeddings'],
                   'type': 'retriever'}},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------
RunLog({'final_outp

# Parallel
- LCEL에서 여러 chain과 같은 작업을 병렬로 실행시킬 수 있는데, 바로 `RunnableParallel` 클래스이다.

In [27]:
from langchain_core.runnables import RunnableParallel
chain1 = ChatPromptTemplate.from_template("What is capital city of {country}?") | model
chain2 = ChatPromptTemplate.from_template("What is the area of ​​{country}?") | model
combined = RunnableParallel(capital=chain1, area=chain2)
combined.invoke({"country": "South Korea"})

{'capital': AIMessage(content='The capital city of South Korea is Seoul ().', id='run-ed7f6388-219b-438d-8e26-ea29a8134bb3-0'),
 'area': AIMessage(content='The area of South Korea is approximately 100,363 square kilometers (38,750 square miles).', id='run-65b740f9-0580-4527-a866-4204b17ba6d4-0')}

In [28]:
# batch를 병렬적으로 처리할 수도 있다.
combined.batch([{"country": "South Korea"}, {"country": "USA"}])

[{'capital': AIMessage(content='The capital city of South Korea is Seoul ().', id='run-7cf475bb-6e65-43a8-9e29-8c36df8295f7-0'),
  'area': AIMessage(content='The area of South Korea is approximately 100,363 square kilometers (38,750 square miles).', id='run-fc4ae244-12dd-4ba0-af52-0caa54685f9f-0')},
 {'capital': AIMessage(content='The capital city of the United States of America is Washington, D.C. (short for District of Columbia).', id='run-e460d9f0-d1ba-4675-89c0-4d699664bb7a-0'),
  'area': AIMessage(content='The total area of the United States of America is approximately 3,805,927 square miles (9,857,306 square kilometers). This includes:\n\n* Land area: 3,717,813 square miles (9,629,091 square kilometers)\n* Water area: 88,114 square miles (228,215 square kilometers)\n\nTo break it down further, the USA is divided into 50 states, and each state has its own unique area. The largest state is Alaska, with an area of approximately 663,300 square miles (1,717,856 square kilometers), whi